# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_POPULATION_RWZI_MUTATIONS]') AND TYPE = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_POPULATION_RWZI_MUTATIONS]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_POPULATION_RWZI_MUTATIONS]'))
CREATE TABLE [VWSSTAGE].[CBS_POPULATION_RWZI_MUTATIONS](
  [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTAGE_CBS_POPULATION_RWZI_MUTATIONS]),
  [RWZI_CODE] [VARCHAR](100) NULL,
  [RWZI_NAAM] [VARCHAR](100) NULL,
  [STARTDATUM] [VARCHAR](100) NULL,
  [EINDDATUM] [VARCHAR](100) NULL,
  [INWONERS] [VARCHAR](100) NULL,
  [REGIO_TYPE] [VARCHAR](100) NULL,
  [REGIO_CODE] [VARCHAR](100) NULL,
  [REGIO_NAAM] [VARCHAR](100) NULL,
  [AANDEEL] [VARCHAR](100) NULL,
  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO


# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 12 * */3 *', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied_mutaties.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied_mutaties.csv'
    ELSE 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied_mutaties.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_Inwoners_per_verzorgingsgebied';
SET @source_description = CONCAT('LOAD: CBS_POPULATION_RWZI_MUTATIONS - ', @workflow_name);
SET @target_name = 'VWSSTAGE.CBS_POPULATION_RWZI_MUTATIONS';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'rwzi_code|rwzi_naam|startdatum|einddatum|inwoners|regio_type|regio_code|regio_naam|aandeel',
    @target_columns = 'RWZI_CODE|RWZI_NAAM|STARTDATUM|EINDDATUM|INWONERS|REGIO_TYPE|REGIO_CODE|REGIO_NAAM|AANDEEL',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_Load_Inwoners_per_verzorgingsgebied_mutaties';
SET @process_description = CONCAT('LOAD: CBS_POPULATION_RWZI_MUTATIONS - ', @workflow_name);
SET @process_source_name = 'SOURCE_Load_Inwoners_per_verzorgingsgebied';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;
GO